In [4]:
import os
import pandas as pd
import numpy as np
import json
from sklearn.metrics import classification_report

def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return(TP, FP, TN, FN)

In [2]:
DATA = pd.DataFrame()
Backend = []
Method = []
Program = []
Prediction = []
Std_err = []
Exp = []
Ground = []
Ideal_exp = []

for files in os.listdir("results/"):
    if "&zne" in files:
        bk,method,configuration = files.replace(".json","").split("&")
    else:
        bk,method = files.replace(".json","").split("&")
        configuration = ""
    
    file = open(f"results/{files}","r")
    result = json.load(file)
    file.close()
    for program in result.keys():
        for mutant in result[program].keys():
            prediction = 1
            std_err = result[program][mutant]["std_err"]
            exp = np.median(result[program][mutant]["mitigated_exps"])
            ideal_exp = result[program][mutant]["ideal_exp"]
            ground = result[program][mutant]["ground"]
            if np.abs(ideal_exp-exp)>std_err:
                prediction=0
            
            Backend.append(bk)
            Method.append(method+"_"+configuration)
            Program.append(program+"_"+mutant)
            Ideal_exp.append(ideal_exp)
            Exp.append(exp)
            Std_err.append(std_err)
            Ground.append(ground)
            Prediction.append(prediction)
    
DATA["Backend"] = Backend
DATA["Method"] = Method
DATA["Program"] = Program
DATA["Ideal_exp"] = Ideal_exp
DATA["Mitigated_exp"] = Exp
DATA["Std_err"] = Std_err
DATA["Ground"] = Ground
DATA["Verdict"] = Prediction

In [3]:
df_group = DATA.groupby(by="Backend")

In [4]:
for name,df in df_group:
    ground = df["Ground"].values
    verdict = df["Verdict"].values
    print(name)
    print("---------------------------------------------")
    print(classification_report(ground,verdict,target_names=["Killed","Survived"],digits=3,zero_division=0))

ibm_brisbane
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.993     0.996      6211
    Survived      0.000     0.000     0.000         0

    accuracy                          0.993      6211
   macro avg      0.500     0.496     0.498      6211
weighted avg      1.000     0.993     0.996      6211

ibm_kyoto
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.991     0.995      6145
    Survived      0.000     0.000     0.000         0

    accuracy                          0.991      6145
   macro avg      0.500     0.495     0.498      6145
weighted avg      1.000     0.991     0.995      6145

ibm_osaka
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.992     0.996      6029
    Survived      0.000     0.000     0.000         0

    accurac

In [5]:
DATA[DATA["Verdict"]==1]

,Backend,Method,Program,Ideal_exp,Mitigated_exp,Std_err,Ground,Verdict
150,ibm_brisbane,cdr_,ghz_indep_qiskit_3.qasm_mutant_38,2.317263,2.317263,4.653624e-04,0,1
160,ibm_brisbane,cdr_,graphstate_indep_qiskit_3.qasm_mutant_28,0.527517,0.527517,4.972350e-04,0,1
330,ibm_brisbane,cdr_,ghz_indep_qiskit_4.qasm_mutant_18,3.746837,3.746837,0.000000e+00,0,1
340,ibm_brisbane,cdr_,graphstate_indep_qiskit_4.qasm_mutant_29,0.405456,0.405456,4.440892e-18,0,1
342,ibm_brisbane,cdr_,graphstate_indep_qiskit_4.qasm_mutant_51,0.392736,0.392736,5.438960e-18,0,1
...,...,...,...,...,...,...,...,...
16709,ibm_osaka,zne_Poly4,qftentangled_indep_qiskit_5.qasm_mutant_315,0.605935,0.600205,2.219694e-02,0,1
16936,ibm_osaka,zne_Poly4,random_indep_qiskit_6.qasm_mutant_355,-0.154988,-0.152502,3.072084e-02,0,1
16998,ibm_osaka,zne_Poly4,ae_indep_qiskit_7.qasm_mutant_479,0.216320,0.229017,3.623080e-02,0,1
17142,ibm_osaka,zne_Poly4,su2random_indep_qiskit_7.qasm_mutant_1026,0.848827,0.868409,2.093835e-02,0,1


In [9]:
df_group = DATA.groupby(by="Method")

In [10]:
for name,df in df_group:
    ground = df["Ground"].values
    verdict = df["Verdict"].values
    print(name)
    print("---------------------------------------------")
    print(classification_report(ground,verdict,target_names=["Killed","Survived"],digits=3,zero_division=0))

cdr_
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.984     0.992      4442
    Survived      0.000     0.000     0.000         0

    accuracy                          0.984      4442
   macro avg      0.500     0.492     0.496      4442
weighted avg      1.000     0.984     0.992      4442

zne_Linear
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.999     0.999      3630
    Survived      0.000     0.000     0.000         0

    accuracy                          0.999      3630
   macro avg      0.500     0.499     0.500      3630
weighted avg      1.000     0.999     0.999      3630

zne_Poly3
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.994     0.997      3583
    Survived      0.000     0.000     0.000         0

    accuracy      

In [3]:
df_group = DATA.groupby(by=["Backend","Method"])
for name,df in df_group:
    ground = df["Ground"].values
    verdict = df["Verdict"].values
    print(name)
    print("---------------------------------------------")
    print(classification_report(ground,verdict,target_names=["Killed","Survived"],digits=3,zero_division=0))

('ibm_brisbane', 'cdr_')
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.986     0.993      1570
    Survived      0.000     0.000     0.000         0

    accuracy                          0.986      1570
   macro avg      0.500     0.493     0.496      1570
weighted avg      1.000     0.986     0.993      1570

('ibm_brisbane', 'zne_Linear')
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.999     1.000      1210
    Survived      0.000     0.000     0.000         0

    accuracy                          0.999      1210
   macro avg      0.500     0.500     0.500      1210
weighted avg      1.000     0.999     1.000      1210

('ibm_brisbane', 'zne_Poly3')
---------------------------------------------
              precision    recall  f1-score   support

      Killed      1.000     0.996     0.998      1191
    Survived 

In [8]:
df_group = DATA.groupby(by=["Backend","Method"])
for name,df in df_group:
    ground = df["Ground"].values
    verdict = df["Verdict"].values
    print(name)
    TP, FN, TN, FP = perf_measure(ground,verdict)
    print(f"TP={TP},FN={FN},TN={TN},FP={FP}")
    print("---------------------------------------------")
    

('ibm_brisbane', 'cdr_')
TP=0,FN=22,TN=1548,FP=0
---------------------------------------------
('ibm_brisbane', 'zne_Linear')
TP=0,FN=1,TN=1209,FP=0
---------------------------------------------
('ibm_brisbane', 'zne_Poly3')
TP=0,FN=5,TN=1186,FP=0
---------------------------------------------
('ibm_brisbane', 'zne_Poly4')
TP=0,FN=15,TN=1015,FP=0
---------------------------------------------
('ibm_brisbane', 'zne_Richard')
TP=0,FN=2,TN=1208,FP=0
---------------------------------------------
('ibm_kyoto', 'cdr_')
TP=0,FN=27,TN=1448,FP=0
---------------------------------------------
('ibm_kyoto', 'zne_Linear')
TP=0,FN=1,TN=1209,FP=0
---------------------------------------------
('ibm_kyoto', 'zne_Poly3')
TP=0,FN=10,TN=1200,FP=0
---------------------------------------------
('ibm_kyoto', 'zne_Poly4')
TP=0,FN=16,TN=1024,FP=0
---------------------------------------------
('ibm_kyoto', 'zne_Richard')
TP=0,FN=3,TN=1207,FP=0
---------------------------------------------
('ibm_osaka', 'cdr_')
TP